# Pre-processing

In [1]:
import pandas as pd

In [4]:
# Read in datasets
income_df = pd.read_csv("datasource-AU_Govt_ABS-UoM_AURIN_DB_3_abs_personal_income_total_income_sa2_2011_2018.csv")
population_df = pd.read_csv("datasource-AU_Govt_ABS-UoM_AURIN_DB_3_abs_regional_population_sa2_2001_2021.csv")
income_support_df = pd.read_csv("datasource-TUA_PHIDU-UoM_AURIN_DB_sa2_incomesupport.csv")

# Drop irrelevant attributes for income dataset
suburb_info = income_df[["sa2_code", "sa2_name"]]
income_df = income_df.loc[:, "sum_aud_2011_12":]
income_df = income_df.join(suburb_info)

# Drop irrelevant attributes for population dataset
estimated_population = population_df.loc[:, "erp_2001":"erp_2021"]
population_df = population_df[["state_name_2016", "sa2_maincode_2016", "sa2_name_2016", "area_km2", 
                               "pop_density_2021_people_per_km2", "births_2016_17", "deaths_2016_17", "births_2017_18", 
                               "deaths_2017_18", "births_2018_19", "deaths_2018_19", "births_2019_20", "deaths_2019_20", 
                               "births_2020_21", "deaths_2020_21", "geom"]]
population_df = population_df.join(estimated_population)

# Drop irrelevant attributes for income support dataset
income_support_df = income_support_df[["area_code", "wkb_geometry", "area_name", "age_pens_3_percent_6_13_6_13", 
                                       "child_welfm_3_percent_6_13_6_13", "disab_pens_3_percent_6_13_6_13", 
                                       "hlth_card_3_percent_6_13_6_13", "pens_cd_hlr_3_percent_6_13_6_13", 
                                       "snr_cd_hldr_3_percent_6_13_6_13", "unemply_ben_3_percent_6_13_6_13", 
                                       "welf_dep_fm_3_percent_6_13_6_13"]]

# Filter instances in Victoria state for population dataset
population_df = population_df[population_df["state_name_2016"] == "Victoria"]
suburb_code = population_df[["sa2_maincode_2016"]]

# Filter instances in Victoria state for income dataset
income_df = income_df.merge(suburb_code, how="inner", left_on="sa2_code", right_on="sa2_maincode_2016").drop("sa2_maincode_2016", axis=1)

# Filter instances in Victoria state for income support dataset
income_support_df = income_support_df.merge(suburb_code, how="inner", left_on="area_code", right_on="sa2_maincode_2016").drop("sa2_maincode_2016", axis=1)